In [1]:
import pathlib
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import anndata
from pybedtools import BedTool

## Parameter

In [2]:
dmr_path = '/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/DMR/axis-mch_rms_results_collapsed.tsv'
dmr_prefix = 'DGmCH'

black_list_path = '/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz'
delta_cutoff=0.3
dms_cutoff = 1

In [3]:
# Parameters
dmr_path = "/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial/ITSpatial_rms_results_collapsed.tsv"
dmr_prefix = "ITSpatial"
black_list_path = "/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz"
skip_quantile = 0.25
delta_to_mean = 0.1
dms_cutoff = 1


## Read DMR

In [4]:
# get column and n_sample
dmr_column = pd.read_csv(dmr_path, sep='\t', nrows=0).columns
n_sample = dmr_column.size - 6
dmr_column = dmr_column.map(lambda i: i[18:] if 'methylation_level_' in i else i)
samples = dmr_column[6:]

dmr_region_infos = []
mc_rates = []
hypo_sig_dict = {sample: [] for sample in samples}
hyper_sig_dict = {sample: [] for sample in samples}

dmr_df = pd.read_csv(dmr_path, sep='\t', skiprows=1, names=dmr_column)
print(dmr_df.shape[0], 'dmr before filter')
dmr_df.index = dmr_df.index.map(lambda i: f'{dmr_prefix}_{i}')

# save raw info
dmr_bed = dmr_df.iloc[:, :4].copy()
mc_rate = dmr_df.iloc[:, 6:].copy()

1588445 dmr before filter


## Save unfiltered dmr info

In [5]:
with pd.HDFStore(pathlib.Path(dmr_path).parent / 'DMRInfo.h5') as hdf:
    hdf['Rate'] = mc_rate
    hdf['Bed'] = dmr_bed

In [6]:
dmr_bed.reset_index().iloc[:, [1, 2, 3, 0]].to_csv(pathlib.Path(dmr_path).parent / 'TotalDMR.nofilter.bed', 
                                                   index=None, header=None, sep='\t')

## Filter DMR

In [7]:
# filter by delta
delta_judge = (mc_rate.max(axis=1) - mc_rate.min(axis=1)) > delta_cutoff
delta_index = mc_rate.index[delta_judge]

print(delta_index.size, 'dmr pass delta filter')

1115675 dmr pass delta filter


In [8]:
# filter by blacklist
black_list = pd.read_csv(black_list_path, sep='\t', header=None)
black_list = BedTool.from_dataframe(black_list)

_dmr_bed = BedTool.from_dataframe(dmr_bed.reset_index().iloc[:, [1,2,3,0]])
filtered_dmr_bed = _dmr_bed.intersect(black_list, v=True).to_dataframe().set_index('name')

white_index = filtered_dmr_bed.index

print(white_index.size, 'dmr pass blacklist filter')

1568410 dmr pass blacklist filter


In [9]:
dmr_df = dmr_df.loc[delta_index & white_index]
dmr_df = dmr_df[dmr_df['number_of_dms'] >= dms_cutoff]
print(dmr_df.shape[0], 'dmr after filter')
dmr_df.head()

1101949 dmr after filter


,#chr,start,end,number_of_dms,hypermethylated_samples,hypomethylated_samples,IT-L23+ACA,IT-L23+AI,IT-L23+MOp,IT-L4+MOs,...,IT-L6+AI,IT-L4+SSp,IT-L6+ACA,IT-L5+ACA,IT-L4+SSs,IT-L6+PFC,IT-L6+SSp,IT-L6+SSs,IT-L6+MOs,IT-L6+MOp
ITSpatial_0,chr1,3006782,3006782,1,NaN,NaN,0.916667,0.857143,0.750000,0.750000,...,0.500000,0.461538,1.000000,0.894737,0.357143,0.714286,0.947368,0.916667,0.857143,0.900000
ITSpatial_2,chr1,3013472,3013579,3,"IT-L6+ACA,IT-L5+ACA,IT-L5+MOp","IT-L23+ORB,IT-L23+SSp,IT-L4+MOp,IT-L23+SSs,IT-...",0.833333,0.728571,0.709220,0.714286,...,0.761905,0.652482,0.974684,0.925170,0.672566,0.888889,0.860294,0.826087,0.941176,0.870229
ITSpatial_3,chr1,3015124,3015124,1,NaN,"IT-L4+SSp,IT-L4+SSs",0.957447,0.941176,0.980000,0.952381,...,1.000000,0.725490,0.942857,0.980000,0.550000,0.960000,0.961538,0.918919,0.916667,0.913043
ITSpatial_4,chr1,3027398,3027514,2,"IT-L5+SSp,IT-L6+ACA,IT-L5+ACA,IT-L6+PFC,IT-L6+MOp","IT-L23+MOp,IT-L23+ORB,IT-L23+SSp,IT-L23+SSs",0.615789,0.419355,0.484472,0.473684,...,0.607143,0.625806,0.764228,0.825758,0.608000,0.694737,0.700637,0.704082,0.670330,0.742857
ITSpatial_6,chr1,3034471,3034471,1,"IT-L5+PFC,IT-L6+SSp,IT-L6+MOp","IT-L23+ACA,IT-L23+SSp,IT-L23+SSs,IT-L4+SSp",0.586207,0.840000,0.684211,0.850000,...,0.833333,0.372881,0.857143,0.822222,0.658537,0.880000,0.932432,0.900000,0.857143,0.957447


## Assign DMR to each sample

In [10]:
# parse results
for sample in samples:
    # here saved a list of np.array
    hypo_index = dmr_df[dmr_df['hypomethylated_samples'].fillna('').apply(lambda i: sample in i)].index
    hyper_index = dmr_df[dmr_df['hypermethylated_samples'].fillna('').apply(lambda i: sample in i)].index
    hypo_sig_dict[sample] = hypo_index
    hyper_sig_dict[sample] = hyper_index
    print(sample, 'HypoDMR ', hypo_index.size, sep='\t')
    print(sample, 'HyperDMR', hyper_index.size, sep='\t')


IT-L23+ACA	HypoDMR 	235168
IT-L23+ACA	HyperDMR	216856
IT-L23+AI	HypoDMR 	161759
IT-L23+AI	HyperDMR	146650
IT-L23+MOp	HypoDMR 	402681
IT-L23+MOp	HyperDMR	149951
IT-L4+MOs	HypoDMR 	177732
IT-L4+MOs	HyperDMR	84391
IT-L5+SSp	HypoDMR 	257749
IT-L5+SSp	HyperDMR	218809
IT-L23+ORB	HypoDMR 	185460
IT-L23+ORB	HyperDMR	213404
IT-L23+PFC	HypoDMR 	209722
IT-L23+PFC	HyperDMR	313843
IT-L23+SSp	HypoDMR 	402779
IT-L23+SSp	HyperDMR	153443
IT-L23+MOs	HypoDMR 	357046
IT-L23+MOs	HyperDMR	151547
IT-L5+PFC	HypoDMR 	149559
IT-L5+PFC	HyperDMR	312176
IT-L5+AI	HypoDMR 	149645
IT-L5+AI	HyperDMR	144496
IT-L5+MOs	HypoDMR 	274407
IT-L5+MOs	HyperDMR	145511
IT-L4+MOp	HypoDMR 	311999
IT-L4+MOp	HyperDMR	215727
IT-L5+MOp	HypoDMR 	283634
IT-L5+MOp	HyperDMR	145594
IT-L5+ORB	HypoDMR 	108744
IT-L5+ORB	HyperDMR	117010
IT-L23+SSs	HypoDMR 	385416
IT-L23+SSs	HyperDMR	122439
IT-L5+SSs	HypoDMR 	169383
IT-L5+SSs	HyperDMR	109701
IT-L6+AI	HypoDMR 	56695
IT-L6+AI	HyperDMR	96779
IT-L4+SSp	HypoDMR 	375274
IT-L4+SSp	HyperDMR	248380
IT-L6

In [11]:
with pd.HDFStore(pathlib.Path(dmr_path).parent / 'DMRInfo.h5') as hdf:
    for sample, hypo_index in hypo_sig_dict.items():
        hdf[f'HypoDMR/{sample}'] = pd.Series(hypo_index)
    for sample, hyper_index in hyper_sig_dict.items():
        hdf[f'HyperDMR/{sample}'] = pd.Series(hyper_index)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L23+ACA'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L23+AI'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L23+MOp'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural nam

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L6+SSs'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L6+MOs'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L6+MOp'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural namin

## Dump DMR bed

In [12]:
hypo_dir = pathlib.Path(dmr_path).parent / 'HypoDMR'
hypo_dir.mkdir(exist_ok=True)
for sample, hypo_index in hypo_sig_dict.items():
    _bed = dmr_bed.loc[hypo_index].reset_index().iloc[:, [1, 2, 3, 0]].to_csv(
        hypo_dir / f'{sample}.DMS{dms_cutoff}.bed', sep='\t', index=None, header=None)

In [13]:
hyper_dir = pathlib.Path(dmr_path).parent / 'HyperDMR'
hyper_dir.mkdir(exist_ok=True)
for sample, hyper_index in hyper_sig_dict.items():
    _bed = dmr_bed.loc[hyper_index].reset_index().iloc[:, [1, 2, 3, 0]].to_csv(
        hyper_dir / f'{sample}.DMS{dms_cutoff}.bed', sep='\t', index=None, header=None)

## DMR hits matrix

In [14]:
sig_dict = hypo_sig_dict

rows = []
cols = []
datas = []
for i, (sample, dmr_index) in enumerate(sig_dict.items()):
    col = dmr_index.map(lambda i: i.split('_')[1]).astype(int).values
    row = (np.ones_like(col) * i).astype(int)
    data = np.ones_like(col)
    rows.append(row)
    cols.append(col)
    datas.append(data)
datas = np.concatenate(datas)
cols = np.concatenate(cols)
rows = np.concatenate(rows)
hits = csr_matrix((datas, (rows, cols)),
                  shape=(mc_rate.shape[1], dmr_bed.shape[0]))

# obs is DMR, var is sample, because all analysis is dmr focused
dmr_hits = anndata.AnnData(X=hits.T,
                           obs=dmr_bed,
                           var=pd.DataFrame([], index=mc_rate.columns))

dmr_hits.write_h5ad(hypo_dir / 'TotalHits.h5ad')


... storing '#chr' as categorical


In [15]:
sig_dict = hyper_sig_dict

rows = []
cols = []
datas = []
for i, (sample, dmr_index) in enumerate(sig_dict.items()):
    col = dmr_index.map(lambda i: i.split('_')[1]).astype(int).values
    row = (np.ones_like(col) * i).astype(int)
    data = np.ones_like(col)
    rows.append(row)
    cols.append(col)
    datas.append(data)
datas = np.concatenate(datas)
cols = np.concatenate(cols)
rows = np.concatenate(rows)
hits = csr_matrix((datas, (rows, cols)),
                  shape=(mc_rate.shape[1], dmr_bed.shape[0]))

# obs is DMR, var is sample, because all analysis is dmr focused
dmr_hits = anndata.AnnData(X=hits.T,
                           obs=dmr_bed,
                           var=pd.DataFrame([], index=mc_rate.columns))

dmr_hits.write_h5ad(hyper_dir / 'TotalHits.h5ad')

... storing '#chr' as categorical
